# 📰 Optional Project: Concurrent News Aggregator

In this project, you'll build a concurrent news aggregator that fetches, filters, and organizes articles.

### What You'll Practice
- Concurrency with `asyncio` and `aiohttp`
- Testing parsing/filtering logic
- Logging errors and info
- Optimization with sets and generators
- Factory and Observer patterns

### Challenge
Add multiple sources and let users subscribe to keywords (Observer pattern).

In [ ]:
import asyncio, aiohttp, logging

logging.basicConfig(filename="news_errors.log", level=logging.ERROR)

class NewsAggregator:
    def __init__(self):
        self.articles = []

    async def fetch(self, session, url):
        try:
            async with session.get(url) as resp:
                return await resp.json()
        except Exception as e:
            logging.error("Error fetching %s: %s", url, e)
            return None

    async def fetch_articles(self, urls):
        async with aiohttp.ClientSession() as session:
            tasks = [self.fetch(session, url) for url in urls]
            results = await asyncio.gather(*tasks)
            for r in results:
                if r:
                    self.articles.extend(r.get("articles", []))

    def filter_articles(self, keyword):
        seen = set()
        filtered = []
        for a in self.articles:
            title = a.get("title", "")
            if keyword.lower() in title.lower() and title not in seen:
                seen.add(title)
                filtered.append(title)
        return filtered

    def save_report(self, keyword, articles):
        with open(f"{keyword}_report.txt", "w") as f:
            for a in articles:
                f.write(a + "\n")
        print(f"Report saved to {keyword}_report.txt")

In [ ]:
# Example run (using placeholder API)
urls = [
    "https://newsapi.org/v2/top-headlines?country=us&apiKey=demo", # replace with real key
]

async def run():
    app = NewsAggregator()
    await app.fetch_articles(urls)
    results = app.filter_articles("python")
    print("Articles found:", results)
    app.save_report("python", results)

# asyncio.run(run())  # Uncomment to execute with a real API